### This script aggregates together geoTIFFs by year into a 4-dimensional HDF5 [time, X, Y, bands]

*(This code should be run in Raijin as paths refer to locations where the geotiffs are)*

#### Import libraries and define bounds for the conversion, destination path for the output files and product and "constants"

*(This example converts Landsat7 data. Similar scripts can be created for other Landsat satellites just modifying the paths to the source data)*

In [ ]:
import numpy as np
import h5py
from osgeo import gdal
import glob
import os.path
from datetime import datetime
import time

years = range(1987, 2015)
lats = range(-31, -25)
lons = range(147, 152)
path = "/g/data/v10/HPCData/test/"
product = "NBAR"

#### Loop over bounds accumulating values in cube ndarray and writing to a HDF5 using chunking and 'lzf' compression

*(Each file contains a cube [1 year, 1 degree, 1 degree, all bands])*
*(units for each dimension are [time epoch, EPSG 4326, EPSG 4326, integer)*

In [ ]:
for lon in lons:
    for lat in lats:
        for year in years:
            if not os.path.isfile(path + "LS7_ETM_NBAR_{0}_{1:04d}_{2}.h5".format(lon, lat, year)):
                if os.path.isdir("/g/data/rs0/tiles/EPSG4326_1deg_0.00025pixel/LS7_ETM/{lon:03d}_{lat:04d}/{year}/".format(lon=lon, lat=lat, year=year)):

                    nbar_tiffs = glob.glob("/g/data/rs0/tiles/EPSG4326_1deg_0.00025pixel/LS7_ETM/{lon:03d}_{lat:04d}/{year}/*" + product + "*.tif".format(lon=lon, lat=lat, year=year))

                    if len(nbar_tiffs) > 0:

                        nbar_tiffs = sorted(nbar_tiffs)
                        with h5py.File(path + 'LS7_ETM_' + product + '_{lon:03d}_{lat:04d}_{year}.h5'.format(lon=lon, lat=lat, year=year), 'w') as h5f:

                            cube = None

                            time_dim = np.zeros(shape=len(nbar_tiffs), dtype=np.float64)

                            for i, tiff in enumerate(nbar_tiffs):
                                print tiff
                                img_params = tiff[:-4].split('/')[-1].split('_')
                                print img_params[5]
                                if len(img_params[5]) == 19:
                                    img_params[5] = img_params[5] + '.000000'
                                d = datetime.strptime(img_params[5], '%Y-%m-%dT%H-%M-%S.%f')
                                posix = time.mktime(d.timetuple()) + (np.float64(d.microsecond)/1000000)
                                time_dim[i] = posix

                                image = gdal.Open(tiff)

                                for band in range(image.RasterCount):
                                    print("{}_{}".format(tiff, band+1))
                                    if band == 0:
                                        band_pack = np.array(image.GetRasterBand(band+1).ReadAsArray())
                                    else:
                                        band_pack = np.dstack((band_pack, np.array(image.GetRasterBand(band+1).ReadAsArray())))

                                if cube is None:
                                    cube = band_pack[np.newaxis,:]
                                else:
                                    cube = np.concatenate((cube, band_pack[np.newaxis,:]), axis=0)
                            
                            ds = h5f.create_dataset(product, data=cube, compression='lzf', chunks=(len(nbar_tiffs), 100, 100, 6))
                            
                            ds.attrs['source'] = "LS7"
                            ds.attrs['axes'] = ["time", "x", "y", "product"]
                            
                            h5f.create_dataset('x', data=np.arange(4000, dtype=np.float64)*0.00025+lon)
                            h5f.create_dataset('y', data=np.arange(4000, dtype=np.float64)*0.00025+lat)
                            h5f.create_dataset('product', data=np.array(['NBAR_0', 'NBAR_1', 'NBAR_2',
                                                                         'NBAR_3', 'NBAR_4', 'NBAR_5'], dtype='|S20'))
                            h5f.create_dataset('time', data=time_dim)
                            
                            ds.dims.create_scale(h5f['x'], "WGS84 Longitude")
                            ds.dims.create_scale(h5f['y'], "WGS84 Latitude")
                            ds.dims.create_scale(h5f['product'], "Product Names")
                            ds.dims.create_scale(h5f['time'], "POSIX Time Since Epoch")

                            ds.dims[0].attach_scale(h5f['time'])
                            ds.dims[0].label('time')
                            ds.dims[1].attach_scale(h5f['x'])
                            ds.dims[1].label('x')
                            ds.dims[2].attach_scale(h5f['y']
                            ds.dims[2].label('y')
                            ds.dims[3].attach_scale(h5f['product'])
                            ds.dims[3].label('product')